In [ ]:
%%capture
pip install tensorflow==2.2.0

In [ ]:
pip install numpy==1.19.5

In [ ]:
import collections
import helper
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

No need for calling project_test.py file as a object and then calling each function by its object. yuou can write all functions directly and then call them directly.

In [ ]:
import numpy as np
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


def _test_model(model, input_shape, output_sequence_length, urdu_vocab_size):
     
    assert model.input_shape == (None, *input_shape[1:]),\
        'Wrong input shape. Found input shape {} using parameter input_shape={}'.format(model.input_shape, input_shape)

    assert model.output_shape == (None, output_sequence_length, urdu_vocab_size),\
        'Wrong output shape. Found output shape {} using parameters output_sequence_length={} and urdu_vocab_size={}'\
            .format(model.output_shape, output_sequence_length, urdu_vocab_size)

    assert len(model.compiled_loss.metrics) >= 0,\
        'No loss function set.  Apply the `compile` function to the model.'

    assert 'sparse_categorical_crossentropy' == model.compiled_loss._losses,\
        'Not using `sparse_categorical_crossentropy` function for loss.'


def test_tokenize(tokenize):
    sentences = [
        'The quick brown fox jumps over the lazy dog .',
        'By Jove , my quick study of lexicography won a prize .',
        'This is a short sentence .']
    tokenized_sentences, tokenizer = tokenize(sentences)
    assert tokenized_sentences == tokenizer.texts_to_sequences(sentences),\
        'Tokenizer returned and doesn\'t generate the same sentences as the tokenized sentences returned. '


def test_pad(pad):
    tokens = [
        [i for i in range4)],
        [i for i in range(6()],
        [i for i in range(3)]]
    padded_tokens = pad(tokens)
    padding_id = padded_tokens[0][-1]
    true_padded_tokens = np.array([
        [i for i in range(4)] + [padding_id]*2,
        [i for i in range(6)],
        [i for i in range(3)] + [padding_id]*3])
    assert isinstance(padded_tokens, np.ndarray),\
        'Pad returned the wrong type.  Found {} type, expected numpy array type.'
    assert np.all(padded_tokens == true_padded_tokens), 'Pad returned the wrong results.'

    padded_tokens_using_length = pad(tokens, 9)
    assert np.all(padded_tokens_using_length == np.concatenate((true_padded_tokens, np.full((3, 3), padding_id)), axis=1)),\
        'Using length argument return incorrect results'


def test_simple_model(simple_model):  
    input_shape = (15096, 4, 1)
    output_sequence_length = 4
    english_vocab_size = 199
    urdu_vocab_size = 344
    model = simple_model(input_shape, output_sequence_length, english_vocab_size, urdu_vocab_size)
    _test_model(model, input_shape, output_sequence_length, urdu_vocab_size)
   

def test_embed_model(embed_model):
    input_shape = (15096, 21)
    output_sequence_length = 21
    english_vocab_size = 199
    urdu_vocab_size = 344

    model = embed_model(input_shape, output_sequence_length, english_vocab_size, urdu_vocab_size)
    _test_model(model, input_shape, output_sequence_length, urdu_vocab_size)

def test_encdec_model(encdec_model):
    input_shape = (15096, 4, 1)
    output_sequence_length = 4
    english_vocab_size = 199
    urdu_vocab_size = 344

    model = encdec_model(input_shape, output_sequence_length, english_vocab_size, urdu_vocab_size)
    _test_model(model, input_shape, output_sequence_length, urdu_vocab_size)


def test_bd_model(bd_model):
    input_shape = (15096, 4, 1)
    output_sequence_length = 4
    english_vocab_size = 199
    urdu_vocab_size = 344

    model = bd_model(input_shape, output_sequence_length, english_vocab_size, urdu_vocab_size)
    _test_model(model, input_shape, output_sequence_length, urdu_vocab_size)


def test_model_final(model_final):
    input_shape = (15096, 4)
    output_sequence_length = 4
    english_vocab_size = 199
    urdu_vocab_size = 344

    model = model_final(input_shape, output_sequence_length, english_vocab_size, urdu_vocab_size)
    _test_model(model, input_shape, output_sequence_length, urdu_vocab_size)



In [ ]:
lines=pd.read_csv('data_new.csv')
english_sentences = lines['english_sentence']
urdu_sentences = lines['urdu_sentence']
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, urdu_sentences[sample_i]))

In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
urdu_words_counter = collections.Counter([word for sentence in urdu_sentences for word in sentence.split()])
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Urdu words.'.format(len([word for sentence in urdu_sentences for word in sentence.split()])))
print('{} unique Urdu words.'.format(len(urdu_words_counter)))
print('10 Most common words in the Urdu dataset:')
print('"' + '" "'.join(list(zip(*urdu_words_counter.most_common(10)))[0]) + '"')

In [ ]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
text_sentences = [
    'The quick brown fox jumps over the lazy dog .','By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

In [ ]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')
test_pad(pad)
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

In [ ]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
# Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk
preproc_english_sentences, preproc_urdu_sentences, english_tokenizer, urdu_tokenizer =\
preprocess(english_sentences, urdu_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_urdu_sequence_length = preproc_urdu_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
urdu_vocab_size = len(urdu_tokenizer.word_index)
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max Urdu sentence length:", max_urdu_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("Urdu vocabulary size:", urdu_vocab_size)

In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('`logits_to_text` function loaded.')

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, urdu_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    loss= 'sparse_categorical_crossentropy'
    optimizer= 'adam'
    metrics = ['accuracy']
    rnn = GRU(32, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(urdu_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    
    model.compile(loss = loss, 
                 optimizer =optimizer, 
                 metrics = metrics)

    return model
input_shape = (15096, 4,1)
output_sequence_length = 4
english_vocab_size = 199
urdu_vocab_size = 344
simple_model(input_shape,output_sequence_length,english_vocab_size,urdu_vocab_size)
tmp_x = pad(preproc_english_sentences, max_urdu_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_urdu_sentences.shape[-2], 1))
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_urdu_sequence_length,
    english_vocab_size,
    urdu_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_urdu_sentences, batch_size=1000, epochs=10, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], urdu_tokenizer))


In [ ]:

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(urdu_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
input_shape = (15096, 4,1)
output_sequence_length = 4
english_vocab_size = 199
urdu_vocab_size = 344
bd_model(input_shape,output_sequence_length,english_vocab_size,urdu_vocab_size)
tmp_x = pad(preproc_english_sentences, preproc_urdu_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_urdu_sentences.shape[-2], 1))
bidi_model = bd_model(
    tmp_x.shape,
    preproc_urdu_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(urdu_tokenizer.word_index)+1)
bidi_model.fit(tmp_x, preproc_urdu_sentences, batch_size=1024, epochs=20, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], urdu_tokenizer))

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, urdu_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(urdu_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
input_shape = (15096, 4)
output_sequence_length = 4
english_vocab_size = 199
urdu_vocab_size = 344
model_final(input_shape,output_sequence_length,english_vocab_size,urdu_vocab_size)
print('Final Model Loaded')

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
 
    tmp_x = pad(preproc_english_sentences, preproc_urdu_sentences.shape[1])
    tmp_x = tmp_x.reshape((-1, preproc_urdu_sentences.shape[-2], 1))
    tmp_x = pad(preproc_english_sentences, max_urdu_sequence_length)
    tmp_x = tmp_x.reshape((-1, preproc_urdu_sentences.shape[-2], 1))
    model = bd_model(
    tmp_x.shape,
    preproc_urdu_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(urdu_tokenizer.word_index)+1)
    
    model.fit(tmp_x, preproc_urdu_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow '
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]]).astype(np.float32)
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_urdu_sentences, english_tokenizer, urdu_tokenizer)